# Advanced Lane Finding Project
## The goals / steps of this project are the following:
* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./writeup_images/image_1_chess_distorsion.png "Distorsion Correction Chessborad"
[image2]: ./writeup_images/image_2_distorsion_straight_lines.png "Distorsion Correction"
[image3]: ./writeup_images/image_3_warped.png "Warped Image"

[image4]: ./writeup_images/image_4_color_1.png
[image5]: ./writeup_images/image_5_color.png
[image6]: ./writeup_images/image_6_color.png

[image7]: ./writeup_images/image_7_thresh.png
[image8]: ./writeup_images/image_8_thresh.png
[image9]: ./writeup_images/image_9_thresh.png
[image10]: ./writeup_images/image_10_thresh.png
[image11]: ./writeup_images/image_11_thresh.png
[image12]: ./writeup_images/image_12_thresh.png
[image13]: ./writeup_images/image_13_thresh.png
[image14]: ./writeup_images/image_14_thresh.png
[image15]: ./writeup_images/image_15_thresh.png
[image16]: ./writeup_images/image_16_thresh.png
[image17]: ./writeup_images/image_17_thresh.png
[image18]: ./writeup_images/image_18_thresh.png
[image19]: ./writeup_images/image_19_thresh.png
[image20]: ./writeup_images/image_20_thresh.png
[image21]: ./writeup_images/image_window_final_1.png
[image22]: ./writeup_images/image_window_final_2.png
[image23]: ./writeup_images/image_window_final_3.png
[image24]: ./writeup_images/final_1.png
[image25]: ./writeup_images/final_2.png
[image26]: ./writeup_images/final_3.png
[image27]: ./writeup_images/final_4.png
[image28]: ./writeup_images/final_5.png
[image29]: ./writeup_images/final_6.png

[video1]: ./project_video_FINAL.mp4 "Video1"
[video2]: ./challenge_video_output_FINAL.mp4 "Video2"


### Camera Calibration

#### 1. Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

The code for this step is as follows:
```python
def camera_calibration(img, objpoints, imgpoints):
    original = img.copy()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        f.subplots_adjust(hspace = .2, wspace=.05)
        #cv2.imwrite('origin'+str(i+1)+'.jpg',original)
        #cv2.imwrite('corners_detected'+str(i+1)+'.jpg',img)
        ax1.imshow(original)
        ax1.set_title('Original Image '+str(i+1), fontsize=30)
        ax2.imshow(img)
        ax2.set_title('Corners detected '+str(i+1),fontsize=30)
 
    return objpoints, imgpoints

def cal_matrix(imge, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, imge.shape[:-1],None,None)
    
    return mtx, dist

def undistort(img,mtx,dist):
    undist = cv2.undistort(img,mtx,dist,None,mtx)
    return undist

objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

#Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for i, fname in enumerate(images):
    img = mpimg.imread(fname)
    objpoints, imgpoints = camera_calibration(img, objpoints, imgpoints)

    
dst = mpimg.imread('camera_cal/calibration1.jpg')
frame = mpimg.imread('test_images/test4.jpg')

mtx, dist = cal_matrix(dst, objpoints, imgpoints)

#Chessboard image Undistortion
udst = undistort(dst, mtx,dist)
#Frame of the Video Undistortion
undistorted = undistort(frame, mtx,dist)

```

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection. This is donde using the `camera_calibration(img, objpoints, imgpoints)` function.

I then used the output `objpoints` and `imgpoints` to compute the camera calibration (mtx) and distortion coefficients (dst) using the `cal_matrix(imge, objpoints, imgpoints)` function. I applied this distortion correction to the test image of the chessboard using the `undistort(img,mtx,dist)` function and obtained this result: 

![alt text][image1]


### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.

To demonstrate this step, I will describe how I apply the distortion correction to one of the test images. As it was descreibed above, once the mtx and dst matrices are calculated, the distrosion correction of the previous image is made using the `undistort(img,mtx,dist)` function and obtained this result: 
![alt text][image2]


#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

I used a combination of color and gradient thresholds to generate a binary image as follows:
```python
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(50,100)):
    # Calculate directional gradient
    # Apply threshold
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if (orient=='x'):
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    elif (orient == 'y'):
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    return grad_binary

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[:,:,2]
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    return mag_binary

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return dir_binary

def img_thresh(img, s_sobel_thresh=(8, 100), sx_thresh=(10, 100)):
    img = np.copy(img)
    ksize = 3
    # Apply each of the thresholding functions
    # Sobel x
    gradx = abs_sobel_thresh(img, orient='x', sobel_kernel=ksize, thresh=(50, 150))
    # Sobel y
    grady = abs_sobel_thresh(img, orient='y', sobel_kernel=ksize, thresh=(50, 150))
    # Magnitud
    mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(30, 100))
    # Dir
    dir_binary = dir_threshold(img, sobel_kernel=ksize, thresh=(0.7, 1.3))

    combined_grad = np.zeros_like(dir_binary)
    combined_grad[((gradx == 1)  & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1

    #def color_thresh_combined(img, s_thresh, l_thresh, v_thresh, b_thresh):
    v_thresh = [230,255]
    s_thresh = [235,255]
    l_thresh = [215,255]
    b_thresh = [230,255]
    lab_b_thresh = [195,255]
    

    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    V_binary = hsv[:,:,2]
    V_binary = V_binary*(255/np.max(V_binary))
    V_thresh_binary= np.zeros_like(V_binary)
    V_thresh_binary[(V_binary >= v_thresh[0]) & (V_binary <= v_thresh[1])] = 1

    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    S_binary = hls[:,:,2]
    max_sat = np.max(S_binary)
    if max_sat >= 245:
        S_binary = S_binary*(210/np.max(S_binary))
    # Threshold x gradient
    S_thresh_binary= np.zeros_like(S_binary)
    S_thresh_binary[(S_binary >= s_thresh[0]) & (S_binary <= s_thresh[1])] = 1


    luv = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    L_binary = luv[:,:,0]
    max_l = np.max(L_binary)
    L_binary = L_binary*(255/np.max(L_binary))
    # Threshold x gradient
    L_thresh_binary= np.zeros_like(L_binary)
    L_thresh_binary[(L_binary >= l_thresh[0]) & (L_binary <= l_thresh[1])] = 1

    lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    LAB_B_binary = lab[:,:,2]
    max_value = np.max(LAB_B_binary)
    if ((max_value <= 190)&((max_l < 252)|(max_sat < 220))):
        if (max_value <= 170):
            LAB_B_binary = LAB_B_binary*(210/np.max(LAB_B_binary))
        else:
            LAB_B_binary = LAB_B_binary*(255/np.max(LAB_B_binary)) 
    lab_B_thresh_binary= np.zeros_like(LAB_B_binary)
    lab_B_thresh_binary[(LAB_B_binary >= lab_b_thresh[0]) & (LAB_B_binary <= lab_b_thresh[1])] = 1
    
    
    B_binary = img[:,:,0]
    max_blue = np.max(B_binary)
    #print(max_blue)
    # Threshold x gradient
    if max_blue <= 238:
        B_binary= B_binary*(255/np.max(B_binary))
    B_thresh_binary = np.zeros_like(B_binary)
    B_thresh_binary[(B_binary >= b_thresh[0]) & (B_binary <= b_thresh[1])] = 1

    color_binary= np.zeros_like(B_binary)
    color_binary[((V_thresh_binary == 1) | (S_thresh_binary == 1) | (L_thresh_binary == 1) | (B_thresh_binary == 1))] = 1
    
   # Sobel x
    sobelx = cv2.Sobel(L_binary, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(S_binary)
    sobel = cv2.Sobel(S_binary , cv2.CV_64F, 1, 0)
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel_s = np.uint8(255*abs_sobel/np.max(abs_sobel))
    s_binary[(scaled_sobel_s >= s_sobel_thresh[0]) & (scaled_sobel_s <= s_sobel_thresh[1])] = 1
    # Stack each channel
    #color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    combined = np.zeros_like(sxbinary)
    combined[((color_binary == 1)|(lab_B_thresh_binary == 1)|((s_binary == 1) & (sxbinary == 1)))] = 1
    
    return combined
```

Regarding color transforms, I use and filter the Value Channel of the HSV color space with a threshold of v_thresh = [230,255], the Saturation Channel of the HLS color space with a threshold of s_thresh = [235,255], the Light Channel of the LUV color space with a threshold of l_thresh = [215,255] and the Blue Channel of the RGB color space with a threshold of b_thresh = [230,255].
    
Here are some examples of my output for this step.

### Image 1
![alt text][image4]
### Image 2
![alt text][image5]
### Image 3
![alt text][image6]

As it can be seen, it works ok for image 1 and 2. However, when the image is dark or shadowed as in Image 3 (from the challenge video), the color spaces thresholds are not enough to find the lanes. Therefore, I use the gradient of the Saturation Channel and the Light Channel with a threshold of s_sobel_thresh = (8, 100) and sx_thresh = (10, 100), respectively.
    
The color `Yellow` of the lines is something we can also take advantage of. In the LAB color space, positive values of B Channel respresent the color yellow. Thus, the B Channel of the LAB color space with a threshold of lab_b_thresh = [195,255] is used.

Here are some examples of my output for this step.

### Image 4
![alt text][image7]
### Image 5
![alt text][image8]
### Image 6
![alt text][image9]
### Image 7
![alt text][image10]
### Image 8
![alt text][image11]
### Image 9
![alt text][image12]
### Image 10
![alt text][image13]
### Image 11
![alt text][image14]
### Image 12
![alt text][image15]
### Image 13
![alt text][image16]
### Image 14
![alt text][image17]
### Image 15
![alt text][image18]
### Image 16
![alt text][image19]
### Image 17
![alt text][image20]


From these images, one can observed that `Light Sobel Threshold` and `Saturation Sobel Threshold` images are really noisy. However, if they are added with an logic `and` operator, as in  `Light & Saturation Thresholds` image, I can easly identify lines in shadowed and dark images.
The `Combined Thresholds` image shows the combiantion of all color sapces, B_channel of the LAB color space and the Light and Saturation Sobel binary images with an `or` operator.

#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

The code for my perspective transform includes a function called `perspective_transform(img, offset = 320)`, which appears as follows:
```python

def perspective_transform(img, offset = 320):
    #define 4 source points src = np.float32([[,],[,],[,],[,]])
    #Note: you could pick any four of the detected corners 
    # as long as those four corners define a rectangle
    #One especially smart way to do this would be to use four well-chosen
    # corners that were automatically detected during the undistortion steps
    #We recommend using the automatic detection of corners in your cod
    src = np.float32([(0.451*img.shape[1], 0.6388*img.shape[0]), (0.1585*img.shape[1], img.shape[0]), (0.88*img.shape[1], img.shape[0]), (0.55*img.shape[1], 0.6388*img.shape[0])])
            # For destination points, I'm arbitrarily choosing some points to be
            # a nice fit for displaying our warped result 
            # again, not exact, but close enough for our purposes
    dst = np.float32([(offset, 0), (offset, img.shape[0]), (img.shape[1]-offset, img.shape[0]), (img.shape[1]-offset, 0)]) # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src,dst)
    inv_M = cv2.getPerspectiveTransform(dst,src)
                # e) use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(img,M,(img.shape[1], img.shape[0]),flags=cv2.INTER_LINEAR)
    return warped, inv_M

```


 The function takes as inputs an image (`img`), as well as the offset to define the destination points (`dst`). Inside the functions is defined the source (`src`) points.  I chose the hardcode the source and destination points in the following manner:
```python
offset= 250
src = np.float32([(0.451*img.shape[1], 0.6388*img.shape[0]),
                  (0.1585*img.shape[1], img.shape[0]), 
                  (0.88*img.shape[1], img.shape[0]), 
                  (0.55*img.shape[1], 0.6388*img.shape[0])])
            # For destination points, I'm arbitrarily choosing some points to be
            # a nice fit for displaying our warped result 
            # again, not exact, but close enough for our purposes
dst = np.float32([(offset, 0), 
                  (offset, img.shape[0]), 
                  (img.shape[1]-offset, img.shape[0]), 
                  (img.shape[1]-offset, 0)]) 
            # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
```

This resulted in the following source and destination points:

| Source        | Destination   | 
|:-------------:|:-------------:| 
| 585, 460      | 320, 0        | 
| 203, 720      | 320, 720      |
| 1127, 720     | 960, 720      |
| 695, 460      | 960, 0        |

I verified that my perspective transform was working as expected by showing the test image and its warped counterpart to verify that the lines appear parallel in the warped image.

![alt text][image3]

#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

The indentification of the lane-line pixels is done by two functions. First, the `find_lane_pixels(binary_warped)` function which code is:
```python

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 30

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        
        win_xleft_low = leftx_current - margin   # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    left_fit, right_fit = (None, None)
    
    # Fit a second order polynomial to each
    if len(leftx) != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit = np.polyfit(righty, rightx, 2)
        
    return left_fit, right_fit, leftx, lefty, rightx, righty, out_img
```
It takes as input the binary_warped image and performs a histogram filter to find the x coordinates of the peaks where there are more pixels. Then, from the bottom to the top of the image, a search is performed through sliding windows. The numeber of windows is preset and the starting points are the x coordinates previously found in the histogram step. During this search, each time that pixels are found, the windows is re-center for the next step. Finnally, I fit my lane lines with a 2nd order polynomial kinda with thw `cv2.polyfit(x,y,grade)` like this:

### Image 1 Window_Search

![all_text][image21]

### Image 2 Window_Search
![all_text][image22]

The second function is used with prior information. Once you have found a polynomial for the lane-lines, it is not necesary to do a blind search. The `search_around_poly(binary_warped, left_fit_search, right_fit_search)` function search around a region defined by the previous polynomial fit and a margin. Its code is:

```python
def search_around_poly(binary_warped, left_fit_search, right_fit_search):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!  
    margin = 80

    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit_search[0]*(nonzeroy**2) + left_fit_search[1]*nonzeroy + 
                    left_fit_search[2] - margin)) & (nonzerox < (left_fit_search[0]*(nonzeroy**2) + 
                    left_fit_search[1]*nonzeroy + left_fit_search[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit_search[0]*(nonzeroy**2) + right_fit_search[1]*nonzeroy + 
                    right_fit_search[2] - margin)) & (nonzerox < (right_fit_search[0]*(nonzeroy**2) + 
                    right_fit_search[1]*nonzeroy + right_fit_search[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Fit new polynomials
    left_fit_new, right_fit_new = (None, None)
    if len(leftx) != 0:
        # Fit a second order polynomial to each
        left_fit_new = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit_new = np.polyfit(righty, rightx, 2)

    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    
    if left_fit_new is not None:
        left_fitx = left_fit_new[0]*ploty**2 + left_fit_new[1]*ploty + left_fit_new[2]
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        
    if right_fit_new is not None:
        right_fitx = right_fit_new[0]*ploty**2 + right_fit_new[1]*ploty + right_fit_new[2]
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))

    # Draw the lane onto the warped blank image
    
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    
    return left_fit_new, right_fit_new, leftx, lefty, rightx, righty, result
```
![alt text][image23]

#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

I did this in lines # through # in my code in `my_other_file.py`

#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

I implemented this step in lines as follows:

```python
def draw_lines(img, inv_M, left_fit, right_fit):
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    margin = 50
    out_img = np.zeros_like(img).astype(np.uint8)
    left_line_window0 = np.array([np.flipud(np.transpose(np.vstack([left_fitx-margin, ploty])))]) 
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))]) 
    right_line_window3 = np.array([np.transpose(np.vstack([right_fitx+margin, ploty]))])                          
    
    central_line_pts = np.hstack((left_line_window1, left_line_window2))
    
    left_side_pts = np.hstack((left_line_window1,left_line_window0))
    right_side_pts = np.hstack((left_line_window2, right_line_window3))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(out_img, np.int_([left_side_pts]), (255,255,0))
    cv2.fillPoly(out_img, np.int_([right_side_pts]), (255,255, 0))
    cv2.fillPoly(out_img, np.int_([central_line_pts]), (0,255, 0))
    warped_image = cv2.warpPerspective(out_img,inv_M,(out_img.shape[1], out_img.shape[0]),flags=cv2.INTER_LINEAR)
    result = cv2.addWeighted(img, 1, warped_image , 0.3, 0)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    return result

def draw_info(img,left_curverad, right_curverad, center_difference, side_position):
    # Display radius of curvature and vehicle offset
    cv2.putText(img, 'Coded by Juan ALVAREZ', (10, 50), cv2.FONT_HERSHEY_PLAIN, 2,
                            (255, 63, 150), 4)
    # Display radius of curvature and vehicle offset
    cv2.putText(img, 'Radius of Curvature of Left line is ' + str(round(left_curverad/1000, 3)) + '(Km)', (10, 100), cv2.FONT_HERSHEY_PLAIN, 2,
                            (255, 63, 150), 4)
    cv2.putText(img, 'Radius of Curvature of Right line is ' + str(round(right_curverad/1000, 3)) + '(Km)', (10, 150), cv2.FONT_HERSHEY_PLAIN, 2,
                            (255, 63, 150), 4)
    cv2.putText(img, 'Vehicle is ' + str(abs(round(center_difference, 3))) + 'm ' + side_position + ' of center', (10, 200), cv2.FONT_HERSHEY_PLAIN, 
                            2, (255, 63, 150), 4) 
    return img
```
I use two functions. The `draw_lines(img, inv_M, left_fit, right_fit)` and `draw_info(img,left_curverad, right_curverad, center_difference, side_position)` functions. 

The fist one perfomrs the inverse perspective transform with the matrix inv_M and plot the lines and the region between lines on the original image. The second one draws the information of my name, the radius of curvature and the position of the car with respect of the center of the camera and the found lines.

Here is an example of my result on a test image:

![alt text][image25]

---

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Here's a [link to my video result of the project video](./project_video_FINAL.mp4)

Here's a [link to my video result of the challenge video](./challenge_video_output_FINAL.mp4)

---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

Here I'll talk about the approach I took, what techniques I used, what worked and why, where the pipeline might fail and how I might improve it if I were going to pursue this project further.  

First, through thresholding color spaces and gradients of the image and combining them, the lines were found. This approach presented problems when the frame was darker and blurrier. Dark shadows represent a huge challenge for my approach. Furthermore, when a car is approaching to the line, it affects the line detections and modifies the fiiting. I also made a line class to keep track the lines during the video, calculate some attributes and methods of the line and perform a sanity check before drawing a new line. The sanity check consist in checking if the lines detected:

* are parallels
* have a similar curvature with respect to the last line detected
* Have a similar horizontal distance to the car

I save the fit values of the lines of the last 5 detections. If a line is not detected, the first of the 5 records is deleted.

The code for the line class is:

```python
class Line():
    
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #plot y
        self.ploty = None
        #coordiantes of base position
        self.base_xy = None
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        self.reset = False
    
    def calculate_radius_of_curvature(self):
        if self.best_fit is not None and self.ploty is not None:
            ym_per_pix = 30/720 # meters per pixel in y dimension
            xm_per_pix = 3.7/700 # meters per pixel in x dimension


            # Define y-value where we want radius of curvature
            # We'll choose the maximum y-value, corresponding to the bottom of the image
            y_eval = np.max(self.ploty)

            if self.allx is not None:
                # Fit new polynomials to x,y in world space
                fit_world = np.polyfit(self.ploty*ym_per_pix, self.bestx*xm_per_pix, 2)

                # Calculate the new radii of curvature
                ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
                radius = (1+(2*fit_world[0]*y_eval+fit_world[1])**2)**(3/2)/(np.absolute(2*fit_world[0]))
        return radius
    
    def update_radius_of_curvature(radius):
        self.radius_of_curvature = radius
        
    def calculate_line_base_pos(self, center):
        self.line_base_pos = self.base_xy[0] - center
    
    def update_base_xy(self):
        y_val = np.max(self.ploty)
        x_val = self.best_fit[0]*y_val**2 + self.best_fit[1]*y_val + self.best_fit[2]
        self.base_xy = (x_val, y_val)
            
    def update_line_fit(self, line_fit, x_coordinates, y_coordinates):
        # add a found fit to the line, up to n
        if line_fit is not None:
                
            if self.best_fit is not None:
                # if we have a best fit, see how this new fit compares
                self.diffs = abs(line_fit-self.best_fit)
                if (self.diffs[0] > 0.001 or \
                   self.diffs[1] > 1 or \
                   self.diffs[2] > 100) and \
                   len(self.current_fit) > 0:
                    # bad fit! abort! abort! ... well, unless there are no fits in the current_fit queue, then we'll take it
                    self.detected = False
                else:
                    self.detected = True
                    
                    self.allx = x_coordinates
                    self.ally = y_coordinates
                    
                    fitx = line_fit[0]*self.ploty**2 + line_fit[1]*self.ploty + line_fit[2]
                    
                    self.recent_xfitted.append(fitx)
                    self.current_fit.append(line_fit)
                    
                    if len(self.current_fit) > 5:
                        # throw out old fits, keep newest n
                        self.current_fit = self.current_fit[len(self.current_fit)-5:]
                        self.recent_xfitted = self.recent_xfitted[len(self.recent_xfitted)-5:]
                    
                    self.best_fit = np.average(self.current_fit, axis=0)
                    radius = self.calculate_radius_of_curvature()
                    self.radius_of_curvature = radius
                    self.update_base_xy()
                    
                    self.bestx = np.average(self.recent_xfitted, axis=0)
            else:
                self.detected = True
                
                self.current_fit = [line_fit]
                
                self.allx = x_coordinates
                self.ally = y_coordinates
                fitx = line_fit[0]*self.ploty**2 + line_fit[1]*self.ploty + line_fit[2]
                self.recent_xfitted = [fitx]
        
                self.bestx = fitx
            
                self.best_fit = line_fit
                radius = self.calculate_radius_of_curvature()
                self.radius_of_curvature = radius
                self.update_base_xy()
            
        # or remove one from the history, if not found
        else:
            self.detected = False
           
            if len(self.current_fit) > 1:
                # delete last line_fit
                self.current_fit = self.current_fit[:len(self.current_fit)-1]
                self.best_fit = np.average(self.current_fit, axis=0)
                self.recent_xfitted = self.recent_xfitted[:len(self.recent_xfitted)-1]
                self.bestx = np.average(self.recent_xfitted, axis=0)

```
The sanity check is done by this code:

```python
#SANITY CHECK
if left_fit is not None and right_fit is not None:
    # calculate x-intercept (bottom of image, x=image_height) for fits
    
    left_fit_bottom = left_fit[0]*height**2 + left_fit[1]*height + left_fit[2]
    right_fit_bottom = right_fit[0]*height**2 + right_fit[1]*height + right_fit[2]
    interception_bottom_difference = abs(right_fit_bottom-left_fit_bottom)

    left_fit_middle = left_fit[0]*(height/6)**2 + left_fit[1]*(height/6) + left_fit[2]
    right_fit_middle = right_fit[0]*(height/6)**2 + right_fit[1]*(height/6) + right_fit[2]
    interception_middle_difference = abs(right_fit_middle-left_fit_middle)

    if (abs(0.43*width - interception_bottom_difference) > 0.15*width) or (abs(0.42*width - interception_middle_difference) > 0.2*width):
        left_fit = None
        right_fit = None

    else:
        if (Left.radius_of_curvature is not None) and (abs(measure_radius_of_curvature(leftx, lefty) - Left.radius_of_curvature) > 3*Left.radius_of_curvature):
            left_fit = None
        if (Right.radius_of_curvature is not None) and (abs(measure_radius_of_curvature(rightx, righty) - Right.radius_of_curvature) > 3*Right.radius_of_curvature):
            right_fit = None 

```

This helped too much to accurately decide if drawing a line or not. A better detection of the lines with more robust methods will help a lot. 